# Tarea 1: Probabilidades - Estimadores - Test de Hipótesis 

#### MDS7101 Estadística: Teoría y Aplicaciones
**Profesora:** Constanza Contreras  
**Auxiliar:** Carolina Salgado  
**Ayudante:** Camila Pulgar  
  
**Estudiantes:** Canela Orellana y Maximiliano Varas


# Parte 1: Planteamiento

### 1. 
**Plantee su pregunta de investigación. Entregue contexto y describa el set de datos que utilizará para su análisis**

lalala esta es la pregunta

Hace años que en Chile se producen incendios forestales cada verano, algunos de los cuales han sido realmente devastadores.

Dado este contexto, se encontró un dataset que contiene los datos de los incendios forestales en Chile durante el año 2017, año durante el cual, los incendios fueron especialmente catastróficos y afectaron muchas zonas del país.

In [5]:
# Paquetes para tratamiento de datos
import pandas as pd
import numpy as np

# Librerías gráficas
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Paquetes estadísticos
from scipy.stats import norm, lognorm, kstest, ttest_ind
import statsmodels.api as sm

# Librerías para machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

### Carga de los datos:

Este dataset contiene 12 columnas, 7779 registros

- Price_CLP: Precio de casa en pesos chilenos (CLP)  
- Price_UF: Precio de casa en unidades de fomento (UF)  
- Price_USD: Precio de casa en dolares americanos (USD)  
- Comuna: Comuna en la que se encuentra la casa.  
- Ubicacion: Ubicación más específica de la casa  
- Dorms: Cuantas habitaciones tiene la casa  
- Baths: Cuantos baños tiene la casa  
- Built Area: Cuantos metros cuadrados construidos tiene la casa  
- Total Area: Cuantos metros cuadrados tiene toda la propiedad  
- Parking: Cuantos estacionamentos tiene la casa  
- id: Codigo de la casa en https://chilepropiedades.cl/ ; Ej: https://chilepropiedades.cl/ver-publicacion/11607213
- Realtor: El corredor de bienes raíces

In [36]:
data = pd.read_csv("precios_casas_rm.csv")
data.head()

,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
0,409285000,11500,509695,QuintaNormal,Hoevel4548y4558,7,4.0,384.0,732.0,3.0,11700213,NaN
1,105000000,2950,130760,PedroAguirreCerda,Rucalhue,2,1.0,112.0,145.0,1.0,10894299,Legales y Propiedades SpA
2,128124000,3600,159557,EstaciónCentral,AvenidaLasParcelas,3,1.0,59.0,243.0,2.0,10257635,Propiedadesrs
3,75000000,2107,93400,Colina,PasajeGonzaloRojas,3,1.0,103.0,73.0,1.0,9232092,Patricio Gajardo propiedades
4,53000000,1489,66002,Colina,HernánDíazArrieta2820,2,1.0,57.0,67.0,1.0,7085397,Patricio Gajardo propiedades


Notar que el dataset no está limpio de valores nulos, por lo que haremos un estudio rápido preliminar del dataset.

In [37]:
# Función que realiza un breve análisis exploratorio.
def exploratory_data_analysis(dataframe):
    dim = dataframe.shape
    print(f'1.- El DataFrame tiene {dim[0]} filas y {dim[1]} columnas\n')
    print(f'2.- El DataFrame esta compuesto por las siguientes columnas: {list(dataframe.columns)}\n')
    print("3.- Ejemplos de filas del DataFrame:\n")
    print('Primeras 5 filas:')
    display(dataframe.head(5))
    print('Últimas 5 filas:')
    display(dataframe.tail(5))
    print('Muestreo aleatorio de 5 filas:')
    display(dataframe.sample(5))
    print("4.- Descripción numérica del Dataframe:\n")
    display(dataframe.describe())
    print('5.- Cantidad de valores nulos por columna:\n')
    print(dataframe.isna().sum(), '\n')
    print('6.- Cantidad de valores únicos por columna:\n')
    print(dataframe.nunique(),'\n')    


In [38]:
exploratory_data_analysis(data)

1.- El DataFrame tiene 7779 filas y 12 columnas

2.- El DataFrame esta compuesto por las siguientes columnas: ['Price_CLP', 'Price_UF', 'Price_USD', 'Comuna', 'Ubicacion', 'Dorms', 'Baths', 'Built Area', 'Total Area', 'Parking', 'id', 'Realtor']

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
0,409285000,11500,509695,QuintaNormal,Hoevel4548y4558,7,4.0,384.0,732.0,3.0,11700213,NaN
1,105000000,2950,130760,PedroAguirreCerda,Rucalhue,2,1.0,112.0,145.0,1.0,10894299,Legales y Propiedades SpA
2,128124000,3600,159557,EstaciónCentral,AvenidaLasParcelas,3,1.0,59.0,243.0,2.0,10257635,Propiedadesrs
3,75000000,2107,93400,Colina,PasajeGonzaloRojas,3,1.0,103.0,73.0,1.0,9232092,Patricio Gajardo propiedades
4,53000000,1489,66002,Colina,HernánDíazArrieta2820,2,1.0,57.0,67.0,1.0,7085397,Patricio Gajardo propiedades


Últimas 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
7774,2491300000,70000,3102491,LasCondes,CalleSanJosédeLaSierra,5,5.0,600.0,1800.0,5.0,4708915,Propiedades Viña Limitada ®
7775,242012000,6800,301385,Peñalolén,PasajeMarNegro,4,2.0,124.0,200.0,1.0,6641660,NaN
7776,3736950000,105000,4653736,LasCondes,CaminoLasFlores/CaminoPiedraRoja,5,7.0,460.0,4925.0,8.0,6032811,Tsi Property
7777,569440000,16000,709141,LaPintana,LosCipreses/LosDuraznos,4,2.0,311.0,2011.0,1.0,5314376,Tsi Property
7778,355828820,9998,443124,Talagante,LucasPacheco/Balmaceda,5,3.0,225.0,366.0,NaN,6186867,Tsi Property


Muestreo aleatorio de 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
3540,765185000,21500,952908,LaReina,ValenzuelaPuelmayOnofreJarpa,4,3.0,145.0,803.0,5.0,11631123,Nexxos
6781,704682000,19800,877562,LaReina,Padrehurtado,5,4.0,260.0,860.0,NaN,9021834,Nativo Propiedades
1230,1131762000,31800,1409417,LoBarnechea,Laslomasdeladehesa,4,5.0,357.0,1003.0,6.0,12038954,Cata Brochet
6473,175000000,4917,217933,Maipú,Elcid,3,3.0,140.0,220.0,3.0,11409757,Todo Propiedades
2856,77000000,2164,95890,PuenteAlto,Notatocoo,4,2.0,75.0,112.0,1.0,6328789,Schumacher Propiedades


4.- Descripción numérica del Dataframe:



,Price_CLP,Price_UF,Price_USD,Dorms,Baths,Built Area,Total Area,Parking,id
count,7.779000e+03,7779.000000,7.779000e+03,7779.000000,7714.000000,7533.000000,7571.000000,5489.000000,7.779000e+03
mean,3.642481e+08,10234.571153,4.536091e+05,3.994087,2.653746,229.923669,807.919826,2.980506,9.910828e+06
std,3.868810e+08,10870.491584,4.817945e+05,1.622821,1.465103,1676.898812,9050.893115,17.749384,2.046317e+06
min,2.085000e+03,0.000000,3.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.213620e+06
25%,1.200000e+08,3372.000000,1.494400e+05,3.000000,2.000000,85.000000,129.500000,1.000000,8.563078e+06
50%,2.050000e+08,5760.000000,2.552930e+05,4.000000,2.000000,128.000000,210.000000,2.000000,1.054807e+07
75%,4.911420e+08,13800.000000,6.116340e+05,5.000000,3.000000,200.000000,443.000000,3.000000,1.152463e+07
max,5.516450e+09,155000.000000,6.869801e+06,27.000000,29.000000,120000.000000,678000.000000,1269.000000,1.234149e+07


5.- Cantidad de valores nulos por columna:

Price_CLP        0
Price_UF         0
Price_USD        0
Comuna           0
Ubicacion        0
Dorms            0
Baths           65
Built Area     246
Total Area     208
Parking       2290
id               0
Realtor        595
dtype: int64 

6.- Cantidad de valores únicos por columna:

Price_CLP     1897
Price_UF      1801
Price_USD     1877
Comuna          51
Ubicacion     6620
Dorms           22
Baths           15
Built Area     573
Total Area    1099
Parking         26
id            7778
Realtor        278
dtype: int64 



Por lo que se propone eliminar las filas que contengan valos nulos. Se utiliza la siguiente función:

In [39]:
#Función que elimina las filas que tengan un nulo.
def clean_data(dataframe):
    dataframe = dataframe.dropna()
    #dataframe['Baths'].astype(np.int64)
    return dataframe.reset_index(drop=True)
data = clean_data(data)

Por lo tanto, el dataset limpio es el siguiente:

In [40]:
data = clean_data(data)
exploratory_data_analysis(data)

1.- El DataFrame tiene 4839 filas y 12 columnas

2.- El DataFrame esta compuesto por las siguientes columnas: ['Price_CLP', 'Price_UF', 'Price_USD', 'Comuna', 'Ubicacion', 'Dorms', 'Baths', 'Built Area', 'Total Area', 'Parking', 'id', 'Realtor']

3.- Ejemplos de filas del DataFrame:

Primeras 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
0,105000000,2950,130760,PedroAguirreCerda,Rucalhue,2,1.0,112.0,145.0,1.0,10894299,Legales y Propiedades SpA
1,128124000,3600,159557,EstaciónCentral,AvenidaLasParcelas,3,1.0,59.0,243.0,2.0,10257635,Propiedadesrs
2,75000000,2107,93400,Colina,PasajeGonzaloRojas,3,1.0,103.0,73.0,1.0,9232092,Patricio Gajardo propiedades
3,53000000,1489,66002,Colina,HernánDíazArrieta2820,2,1.0,57.0,67.0,1.0,7085397,Patricio Gajardo propiedades
4,75000000,2107,93400,Colina,GabrielaMistral/PabloNeruda,3,1.0,70.0,80.0,2.0,11688480,Patricio Gajardo propiedades


Últimas 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
4834,85060100,2390,105928,SanBernardo,GeneralUrrutia,5,1.0,120.0,76.0,1.0,6959129,V & S Asesorias & Gestion Inmobiliaria
4835,1352420000,38000,1684209,LoBarnechea,CaminoLaGolondrina,5,6.0,500.0,913.0,4.0,4708978,Propiedades Viña Limitada ®
4836,2491300000,70000,3102491,LasCondes,CalleSanJosédeLaSierra,5,5.0,600.0,1800.0,5.0,4708915,Propiedades Viña Limitada ®
4837,3736950000,105000,4653736,LasCondes,CaminoLasFlores/CaminoPiedraRoja,5,7.0,460.0,4925.0,8.0,6032811,Tsi Property
4838,569440000,16000,709141,LaPintana,LosCipreses/LosDuraznos,4,2.0,311.0,2011.0,1.0,5314376,Tsi Property


Muestreo aleatorio de 5 filas:


,Price_CLP,Price_UF,Price_USD,Comuna,Ubicacion,Dorms,Baths,Built Area,Total Area,Parking,id,Realtor
2352,68000000,1911,84682,PuenteAlto,SanGuillermo/VientoNorte,3,1.0,80.0,70.0,1.0,6799327,Procasa Adrián Ulloa Propiedades
4161,110862850,3115,138061,SanBernardo,Ministeripascual,3,2.0,50.0,122.0,2.0,6560948,Grupo Premium Propiedades Santiago
3335,480465000,13500,598337,SanBernardo,Elsendero,4,4.0,450.0,2000.0,4.0,11533179,Easyprop
4545,63000000,1770,78456,SanBernardo,LoBlancoconSanFrancisco,3,1.0,50.0,118.0,3.0,8210992,Unne
4833,188627000,5300,234903,EstaciónCentral,LaCoruña5044,4,2.0,114.0,220.0,3.0,4822161,Promoción Inmobiliaria


4.- Descripción numérica del Dataframe:



,Price_CLP,Price_UF,Price_USD,Dorms,Baths,Built Area,Total Area,Parking,id
count,4.839000e+03,4839.000000,4.839000e+03,4839.000000,4839.000000,4839.000000,4839.000000,4839.000000,4.839000e+03
mean,3.972957e+08,11163.133292,4.947642e+05,4.010953,2.804918,238.529035,865.452366,3.104980,9.985872e+06
std,4.000722e+08,11241.133216,4.982219e+05,1.466902,1.479908,1881.661443,10878.837230,18.892391,2.001661e+06
min,4.100000e+03,0.000000,5.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.367271e+06
25%,1.300000e+08,3653.000000,1.618930e+05,3.000000,2.000000,89.000000,134.000000,1.000000,8.662740e+06
50%,2.439694e+08,6855.000000,3.038220e+05,4.000000,3.000000,132.000000,227.000000,2.000000,1.071066e+07
75%,5.338500e+08,15000.000000,6.648190e+05,5.000000,4.000000,208.500000,480.000000,3.000000,1.153714e+07
max,4.804650e+09,135000.000000,5.983375e+06,25.000000,29.000000,120000.000000,678000.000000,1269.000000,1.234149e+07


5.- Cantidad de valores nulos por columna:

Price_CLP     0
Price_UF      0
Price_USD     0
Comuna        0
Ubicacion     0
Dorms         0
Baths         0
Built Area    0
Total Area    0
Parking       0
id            0
Realtor       0
dtype: int64 

6.- Cantidad de valores únicos por columna:

Price_CLP     1425
Price_UF      1364
Price_USD     1413
Comuna          49
Ubicacion     4133
Dorms           18
Baths           14
Built Area     505
Total Area     929
Parking         26
id            4838
Realtor        246
dtype: int64 



Probando algunos gráficos interesantes:

In [57]:
fig = px.scatter(data, 
                 x="Comuna", 
                 y="Price_CLP",
                 color="Dorms",
                 color_continuous_scale="viridis_r")
fig.show()

In [55]:
fig = px.histogram(data, x="Price_CLP",marginal='box')
fig.show()

In [54]:
fig = px.scatter_matrix(
    data,
    height=1000,
    dimensions=['Price_UF', 'Dorms', 'Baths', 'Built Area', 'Total Area', 'Parking',
    ],
    # symbol="species",
    # color="species",
)
fig.show()

### 2. 
**Seleccione la (o las) variables que considere le permiten caracterizar el fenómeno a estudiar y grafíquela(s)**

las variables a que caracterizan el fenómenose encontrarán según...

In [ ]:
#Cosas que hacemos para ver las variables que caracterizan el fenómeno
#Quizás no hay que hacer nada y es pura tincada

Los gráficos que son interesantes de analizar son...

In [ ]:
#Gráficos de las variables

### 3.
**Explique el (los) gráficos realizados, ¿Qué distribución sigue el fenómeno?, ¿Qué factores
podrían estar explicando este comportamiento?**

lalala gráficos.

Se observa que la distribución es similar a una...

lo cual se podría deber a.... 

In [ ]:
#En vola un gráfico de comparación, superponiendo la distribución que decimos que se parece
#y la observada en los datos

### 4.
**Plantee una (o varias) distribuciones que serían buenas candidatas para modelar lo que está estudiando. Justifique su elección. Estime los parámetros de esa (o esas) distribución y analice los resultados del modelo planteado con respecto a los datos. ¿Cómo es el ajuste? ¿Cuáles factores podrían explicar las diferencias entre los datos y el modelo?**

No entiendo la diferencia entre esta parte y la anterior D:

# Parte 2: Análisis

**Para esta parte se espera que profundice en los resultados obtenidos en la parte 1, relacionándolos con los conceptos que hemos visto en el curso. Todo esto por medio de un análisis exhaustivo de su problema y su entendimiento de los contenidos revisados en clases. Dentro de ese análisis, responda: ¿Entre qué rangos se mueve su estimación? ¿Qué tan satisfecho está con esta estimación? ¿Por qué? ¿Qué ocurre con las variables que podrían incidir en su resultado? Presente cálculos si puede hacerlo o justificaciones/fundamentos teóricos si no tiene datos para caracterizarlas.**

# Parte 3: Analizando resultados de un modelo de ciencia de datos

### 1.
**Escoja un modelo clásico de ciencia de datos que sea comúnmente utilizado para abordar la problemática que está estudiando. Aplíquelo en sus datos y presente sus resultados. Si lo desea puede utilizar distintas configuraciones o modelos para entrenar con sus datos. Presente las métricas comunes para evaluar la calidad de su modelo. Explique y comente.**

In [ ]:
#Entrenar con una grid?

In [ ]:
#Métricas

In [ ]:
#Gráficos

Explicamos

Comentamos

### 2. 
**Utilizando la siguiente función:
$$Bias=E[\hat{\Theta}]-\Theta$$
Analice el trade-off de sesgo y varianza presente en sus resultados. ¿Es coherente lo descrito por las métricas con que evaluó su modelo de ciencia de datos con lo obtenido por esta función? Analice y comente los resultados obtenidos, si son coherentes con la teoría y a qué explicaría lo obtenido.**

In [ ]:
#Función

Comentarios sobre coherencia y blabla